In [42]:
import pandas as pd
import numpy as np
import string
import re
import random 
import os
import csv

from gensim.models.word2vec import Word2Vec

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import metrics
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from imblearn.ensemble import BalancedRandomForestClassifier
import nltk

## for word embedding
import gensim
import gensim.downloader as gensim_api
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt

# Tf-idf RF


In [45]:
auc_scores = []
pr_auc = []

dat = pd.read_csv("20220115_all_paragraphs_2020_added.csv")
annotated = pd.read_csv("All_annotated_data_exclusion_LO_recodedd.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # pipeline BOW, tfidf and RF
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', RandomForestClassifier(n_estimators=5000, max_depth=3,max_features='sqrt', random_state=0, class_weight="balanced")),
                                                 ])
    ## Fit the model to the training set
    text_clf = text_clf.fit(train['text'], train['pred_class'])
    ## Predict out-of-sample on the test set and compute AUC
    preds = text_clf.predict_proba(test['text'])
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(test['true_class'], preds[:,1], pos_label=1)
    auc_scores = auc_scores + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(test['true_class'],  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    print("PR AUC: " + str(lr_auc))
    
print("Mean AUC: " + str(np.mean(auc_scores)))
print("Mean PR AUC: " + str(np.mean(pr_auc)))

confusion = confusion_matrix(test['true_class'],text_clf.predict(test['text']) )
print(confusion)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AUC: 0.8072892938496584
PR AUC: 0.16302770766282285
AUC: 0.7131818181818181
PR AUC: 0.22040660343565313
AUC: 0.8086363636363636
PR AUC: 0.06949614583647451
AUC: 0.6924829157175398
PR AUC: 0.2294438174535073
AUC: 0.4381818181818182
PR AUC: 0.027598573239031858
AUC: 0.8104545454545455
PR AUC: 0.5505564537768415
AUC: 0.8440909090909091
PR AUC: 0.10832624777477803
AUC: 0.8277272727272728
PR AUC: 0.22490187853656177
AUC: 0.7398633257403189
PR AUC: 0.11090422993664077
AUC: 0.7104545454545454
PR AUC: 0.4096055695190565
AUC: 0.7372727272727272
PR AUC: 0.06398217678174048
AUC: 0.6904545454545455
PR AUC: 0.411083053192737
AUC: 0.9325740318906606
PR AUC: 0.6206149406149406
AUC: 0.9427272727272726
PR AUC: 0.33416623868685724
AUC: 0.7263636363636363
PR AUC: 0.24823054454054208
AUC: 0.6031818181818182
PR AUC: 0.06246345019812863
AUC: 0.6177272727272727
PR AUC: 0.2628521338107989
AUC: 0.6927272727272727
PR AUC: 0.2168243656340853
AUC: 0.849090909090909
PR AUC: 0.26325078479025843
AUC: 0.8777272727272

In [46]:
with open("exclusion_pr_auc_tfidf.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("exclusion_auc_tfidf.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores:
        csvwriter.writerow([row])

#  Local Word2Vec RF

In [57]:
auc_scores = []
pr_auc = []

dat = pd.read_csv("20220115_all_paragraphs_2020_added.csv")
annotated = pd.read_csv("All_annotated_data_round_1_LO_recodedd_20220209.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")
avg_embeddings = loadtxt('avg_embeddings.csv', delimiter=',')

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # attach train/test set to original df
    train_id = train['speech_par_id']
    test_id = test['speech_par_id']    
    dat['test'] = np.where(dat['speech_par_id'].isin(test_id), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train_id), 1, 0)
    # get the embeddings to train/test set
    train_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(train_id)].index.tolist()])
    test_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(test_id)].index.tolist()])
    ## Fit the model to the training set
    text_clf = RandomForestClassifier(n_estimators=500, max_depth=3, random_state=0, max_features = "sqrt", class_weight="balanced")
    text_clf.fit(train_vec,  dat[dat['train']==1].pred_class) # use the original df to match the order
    ## Predict out-of-sample on the test set and compute AUC
    preds = text_clf.predict_proba(test_vec)
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores = auc_scores + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    print("PR AUC: " + str(lr_auc))
    
print("Mean AUC: " + str(np.mean(auc_scores)))
print("Mean PR AUC: " + str(np.mean(pr_auc)))

confusion = confusion_matrix(dat[dat['test']==1].pred_class,text_clf.predict(test_vec) )
print(confusion)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AUC: 0.6158730158730158
PR AUC: 0.015636972002978325
AUC: 0.5004545454545455
PR AUC: 0.20920039050125813
AUC: 0.7024943310657596
PR AUC: 0.0556115056729728
AUC: 0.781489841986456
PR AUC: 0.12246772996896844
AUC: 0.5148648648648648
PR AUC: 0.21570140836014381
AUC: 0.902262443438914
PR AUC: 0.3011859379201029
AUC: 0.511617312072893
PR AUC: 0.2707537489071317
AUC: 0.6308390022675736
PR AUC: 0.17013016150947183
AUC: 0.7040909090909091
PR AUC: 0.24687552802729268
AUC: 0.8710407239819005
PR AUC: 0.0820941758796296
AUC: 0.650113895216401
PR AUC: 0.056536453122667814
AUC: 0.5682539682539682
PR AUC: 0.21368109812017871
AUC: 0.7325791855203619
PR AUC: 0.02629850934361664
AUC: 0.8824324324324324
PR AUC: 0.12620695765058096
AUC: 0.7054545454545454
PR AUC: 0.03544204648296536
AUC: 0.6988713318284425
PR AUC: 0.024945542386272477
AUC: 0.8439909297052155
PR AUC: 0.3038578927841468
AUC: 0.4444444444444444
PR AUC: 0.011566276068572991
AUC: 0.4303854875283447
PR AUC: 0.012812532256351282
AUC: 0.682539682

In [58]:
with open("exclusion_pr_auc_w2v.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("exclusion_auc_w2v.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores:
        csvwriter.writerow([row])

# Pretrained Word2Vec

In [59]:
auc_scores = []
pr_auc = []

dat = pd.read_csv("20220131_all_paragraphs_2020_added_missings_added.csv")
annotated = pd.read_csv("All_annotated_data_round_1_LO_recodedd_20220209.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")
avg_embeddings = loadtxt('avg_embeddings_pretrained.csv', delimiter=',')

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
# attach train/test set to original df
    train_id = train['speech_par_id']
    test_id = test['speech_par_id']    
    dat['test'] = np.where(dat['speech_par_id'].isin(test_id), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train_id), 1, 0)
    # get the embeddings to train/test set
    train_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(train_id)].index.tolist()])
    test_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(test_id)].index.tolist()])
    ## Fit the model to the training set
    text_clf = RandomForestClassifier(n_estimators=5000, max_depth=10, random_state=0, max_features = "sqrt", class_weight="balanced")
    text_clf.fit(train_vec,  dat[dat['train']==1].pred_class) # use the original df to match the order
    ## Predict out-of-sample on the test set and compute AUC
    preds = text_clf.predict_proba(test_vec)
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores = auc_scores + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    print("PR AUC: " + str(lr_auc))
    
print("Mean AUC: " + str(np.mean(auc_scores)))
print("Mean PR AUC: " + str(np.mean(pr_auc)))

confusion = confusion_matrix(dat[dat['test']==1].pred_class,text_clf.predict(test_vec) )
print(confusion)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AUC: 0.4616780045351474
PR AUC: 0.009621015629519013
AUC: 0.6486363636363637
PR AUC: 0.21221819621911805
AUC: 0.7029478458049888
PR AUC: 0.0720849208235716
AUC: 0.7927765237020317
PR AUC: 0.06287983187815918
AUC: 0.6454954954954955
PR AUC: 0.02777717546086555
AUC: 0.8981900452488688
PR AUC: 0.08915684817256145
AUC: 0.5981776765375855
PR AUC: 0.03976493149732508
AUC: 0.6585034013605442
PR AUC: 0.08741941897695911
AUC: 0.49363636363636365
PR AUC: 0.059290488590597185
AUC: 0.7099547511312218
PR AUC: 0.031387582220651064
AUC: 0.7211845102505695
PR AUC: 0.03217053404245808
AUC: 0.5968253968253968
PR AUC: 0.06143223574119545
AUC: 0.8018099547511313
PR AUC: 0.08339443968534008
AUC: 0.8265765765765766
PR AUC: 0.05387634749085301
AUC: 0.6595454545454545
PR AUC: 0.01772159068907929
AUC: 0.6406320541760723
PR AUC: 0.016360025104828233
AUC: 0.6167800453514739
PR AUC: 0.02497618230296804
AUC: 0.417687074829932
PR AUC: 0.009255086060898395
AUC: 0.5424036281179139
PR AUC: 0.018500427287264816
AUC: 0.

In [60]:
with open("exclusion_pr_auc_pretrained.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("exclusion_auc_pretrained.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores:
        csvwriter.writerow([row])

# RFC

In [61]:
auc_scores_d2v = []
pr_auc = []
accuracy_scores_d2v = []

dat = pd.read_csv("20201115_all_paragraphs.csv")
dat['speech_par_id'] = dat['Speech_id'].astype(str) + "_" + dat['par_id'].astype(str)

model = Word2Vec.load("doc2vec_wordvecs.model") 
annotated = pd.read_csv("All_annotated_data_round_1_LO_recodedd_20220209.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # merge test and train sets back onto the total df
    dat['test'] = np.where(dat['speech_par_id'].isin(test["speech_par_id"]), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train["speech_par_id"]), 1, 0)
    # so that teh order of rows are the same with the embeddings
    test_set = np.asarray([model.docvecs[i] for i in dat[dat['test'] == 1].index.tolist()])
    train_set = np.asarray([model.docvecs[i] for i in dat[dat['train']== 1].index.tolist()])
    ## Initialize a random forest classifier
    gbc = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=0, max_features = "sqrt", class_weight="balanced")
    ## Fit the model to the training set
    gbc.fit(train_set, dat[dat['train']==1].pred_class)
    ## Predict out-of-sample on the test set and compute AUC
    preds = gbc.predict_proba(test_set)
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores_d2v = auc_scores_d2v + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    accuracy_d2v = metrics.accuracy_score(dat[dat['test']==1].pred_class, gbc.predict(test_set))
    accuracy_scores_d2v = accuracy_scores_d2v + [accuracy_d2v]
    print("Accuracy: " + str(accuracy_d2v))
    print("PR_AUC: " + str(lr_auc))

print("Mean AUC: " + str(np.mean(auc_scores_d2v)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_d2v)))

confusion = confusion_matrix(dat[dat['test']==1].pred_class, gbc.predict(test_set))
print(confusion)

AUC: 0.6
Accuracy: 0.9887892376681614
PR_AUC: 0.013346555667979964
AUC: 0.5895454545454546
Accuracy: 0.9887640449438202
PR_AUC: 0.019469279947893034
AUC: 0.7482993197278911
Accuracy: 0.9887892376681614
PR_AUC: 0.03282569317496062
AUC: 0.8470588235294118
Accuracy: 0.9888143176733781
PR_AUC: 0.09208363797129727
AUC: 0.534841628959276
Accuracy: 0.9888143176733781
PR_AUC: 0.014122313564013047
AUC: 0.5923076923076924
Accuracy: 0.9888143176733781
PR_AUC: 0.07498055752719385
AUC: 0.6022779043280182
Accuracy: 0.9887387387387387
PR_AUC: 0.0475412924776617
AUC: 0.8263038548752835
Accuracy: 0.9887892376681614
PR_AUC: 0.1305141384352742
AUC: 0.8132118451025058
Accuracy: 0.9887387387387387
PR_AUC: 0.10013113707934782
AUC: 0.818552036199095
Accuracy: 0.9888143176733781
PR_AUC: 0.03247700323200654
AUC: 0.7403189066059225
Accuracy: 0.9887387387387387
PR_AUC: 0.022298612132455157
AUC: 0.7206818181818182
Accuracy: 0.9887640449438202
PR_AUC: 0.03127681737313126
AUC: 0.7742081447963801
Accuracy: 0.9888143

In [63]:
with open("exclusion_pr_auc.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("exclusion_auc.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores_d2v:
        csvwriter.writerow([row])

In [62]:
print("Mean ROC-AUC: " + str(np.mean(auc_scores_d2v)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_d2v)))
print("Mean PR_AUC: " + str(np.mean(pr_auc)))


Mean ROC-AUC: 0.7145048115735873
Mean Accuracy: 0.9887881578480556
Mean PR_AUC: 0.05079569478099346


# balanced RFC

In [64]:
auc_scores_balanced = []
pr_auc_balanced = []
accuracy_scores_balanced = []

dat = pd.read_csv("20201115_all_paragraphs.csv")
dat['speech_par_id'] = dat['Speech_id'].astype(str) + "_" + dat['par_id'].astype(str)
# X = np.asarray([model.docvecs[i] for i in label_dat.index.tolist()])
# Y = np.asarray(label_dat['label'].tolist(), dtype="int")
model = Word2Vec.load("doc2vec_wordvecs.model") 
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")


for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/"):
    ## load the test set 
    annotated = pd.read_csv("All_annotated_data_round_1_LO_recodedd_20220209.csv")
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # merge test and train sets back onto the total df
    dat['test'] = np.where(dat['speech_par_id'].isin(test["speech_par_id"]), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train["speech_par_id"]), 1, 0)
    # so that teh order of rows are the same with the embeddings
    test_set = np.asarray([model.docvecs[i] for i in dat[dat['test'] == 1].index.tolist()])
    train_set = np.asarray([model.docvecs[i] for i in dat[dat['train']== 1].index.tolist()])
    ## Initialize a random forest classifier
    brfc = BalancedRandomForestClassifier(n_estimators=5000, max_depth=3, random_state=0, max_features = "sqrt")
    ## Fit the model to the training set
    brfc.fit(train_set, dat[dat['train']==1].pred_class)
    ## Predict out-of-sample on the test set and compute AUC
    preds = brfc.predict_proba(test_set)
    fpr, tpr, thresholds = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores_balanced = auc_scores_balanced + [metrics.auc(fpr, tpr)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc_balanced = pr_auc_balanced + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr, tpr)))
    accuracy_balanced = metrics.accuracy_score(dat[dat['test']==1].pred_class, brfc.predict(test_set))
    accuracy_scores_balanced = accuracy_scores_balanced + [accuracy_balanced]
    print("Accuracy: " + str(accuracy_balanced))
    print("PR_AUC: " + str(lr_auc))

print("Mean AUC: " + str(np.mean(auc_scores_balanced)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_balanced)))

AUC: 0.5410430839002268
Accuracy: 0.6300448430493274
PR_AUC: 0.011937102139544867
AUC: 0.5936363636363636
Accuracy: 0.647191011235955
PR_AUC: 0.028180540722016312
AUC: 0.7954648526077097
Accuracy: 0.5874439461883408
PR_AUC: 0.04111763482320745
AUC: 0.8479638009049772
Accuracy: 0.5413870246085011
PR_AUC: 0.2832899236615389
AUC: 0.5538461538461539
Accuracy: 0.697986577181208
PR_AUC: 0.015058878569845845
AUC: 0.6380090497737557
Accuracy: 0.6085011185682326
PR_AUC: 0.2119500602700287
AUC: 0.6779043280182232
Accuracy: 0.5990990990990991
PR_AUC: 0.21297471688537686
AUC: 0.7682539682539683
Accuracy: 0.5717488789237668
PR_AUC: 0.3366219489152245
AUC: 0.801366742596811
Accuracy: 0.5743243243243243
PR_AUC: 0.2806327673067278
AUC: 0.7868778280542987
Accuracy: 0.5659955257270693
PR_AUC: 0.029039062242195223
AUC: 0.7284738041002278
Accuracy: 0.5292792792792793
PR_AUC: 0.053057700351859735
AUC: 0.7390909090909091
Accuracy: 0.6
PR_AUC: 0.02248673903449239
AUC: 0.8407239819004526
Accuracy: 0.604026845

In [66]:
with open("exclusion_pr_auc_balanced.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc_balanced:
        csvwriter.writerow([row])
    
with open("exclusion_auc_balanced.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores_balanced:
        csvwriter.writerow([row])

In [65]:
print("Mean ROC-AUC: " + str(np.mean(auc_scores_balanced)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_balanced)))
print("Mean PR_AUC: " + str(np.mean(pr_auc_balanced)))


Mean ROC-AUC: 0.7139847021391479
Mean Accuracy: 0.5905306666366167
Mean PR_AUC: 0.09118638504971521


In [55]:
not_aligned = []
for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_exclusion/" + "/" + filename
    test = pd.read_csv(file)
    for id in test['speech_par_id']:
        if id in dat['speech_par_id']:
            if dat[dat['speech_par_id']== id]['text'].values[0] != test[test["speech_par_id"] == id]['text'].values[0]:
                print(dat[dat['speech_par_id']== id])
                print(test[test["speech_par_id"] == id])
                not_aligned.append(dat[dat['speech_par_id']== id])

In [56]:
## check the annotation data
dat = pd.read_csv("20220131_all_paragraphs_2020_added_missings_added.csv")
not_aligned = pd.DataFrame(columns = dat.columns)
for id in annotated['speech_par_id']:
    if id in dat['speech_par_id'].tolist():
        if dat.loc[dat['speech_par_id']== id,'text'].values[0] != annotated.loc[annotated["speech_par_id"] == id, 'text'].values[0]:
            print(dat[dat['speech_par_id']== id])
            print(annotated[annotated["speech_par_id"] == id])
            not_aligned = not_aligned.append(dat[dat['speech_par_id']== id])

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


       Unnamed: 0 Speech_id  \
12231       12232      2123   

                                                    text party    term  comp  \
12231  Mr. Nixon has taken the American public for gr...   dem  1968.0  True   

      populist_old_keywords  par_id speech_par_id  
12231                 False      15       2123_15  
                                                text  pred_class  \
0  Mr. Nixon has taken the American public for gr...           0   

  speech_par_id                          speech_par_id_old  
0       2123_15  2008-10-11-philadelphia-pennsylvania-2_10  
       Unnamed: 0 Speech_id  \
39387       39388      1619   

                                                    text party    term   comp  \
39387  In any case, I come tonight and ask your suppo...   dem  1960.0  False   

      populist_old_keywords  par_id speech_par_id  
39387                  True      14       1619_14  
                                                text  pred_class  \
1  In any case,

      Unnamed: 0 Speech_id                                               text  \
9346        9347      1367  And, you know, he said that Eisenhower had had...   

     party    term   comp populist_old_keywords  par_id speech_par_id  
9346   dem  1956.0  False                 False       9        1367_9  
                                                  text  pred_class  \
159  And, you know, he said that Eisenhower had had...           0   

    speech_par_id speech_par_id_old  
159        1367_9            1367_9  
       Unnamed: 0 Speech_id  \
38812       38813      1369   

                                                    text party    term  comp  \
38812  When there was a strike at the auto agency of ...   dem  1956.0  True   

      populist_old_keywords  par_id speech_par_id  
38812                 False       5        1369_5  
                                                  text  pred_class  \
160  When there was a strike at the auto agency of ...           0   

    spe

       Unnamed: 0 Speech_id  \
19608       19609      1675   

                                                    text party    term   comp  \
19608  Mr. Nixon and I disagree on what is the best i...   dem  1960.0  False   

      populist_old_keywords  par_id speech_par_id  
19608                 False       1        1675_1  
                                                  text  pred_class  \
257  Mr. Nixon and I disagree on what is the best i...           0   

    speech_par_id speech_par_id_old  
257        1675_1            1675_1  
       Unnamed: 0 Speech_id  \
24495       24496      1688   

                                                    text party    term   comp  \
24495  Distinguished guests, members of the Congress,...   dem  1960.0  False   

      populist_old_keywords  par_id speech_par_id  
24495                 False       0        1688_0  
                                                  text  pred_class  \
259  Distinguished guests, members of the Congress,..

       Unnamed: 0 Speech_id  \
34005       34006      1906   

                                                    text party    term   comp  \
34005  The statue atop the State House of Rhode Islan...   dem  1964.0  False   

      populist_old_keywords  par_id speech_par_id  
34005                 False      16       1906_16  
                                                  text  pred_class  \
386  The statue atop the State House of Rhode Islan...           0   

    speech_par_id speech_par_id_old  
386       1906_16           1906_16  
       Unnamed: 0 Speech_id  \
32675       32676      1907   

                                                    text party    term   comp  \
32675  For 175 years, through more than forty nationa...   dem  1964.0  False   

      populist_old_keywords  par_id speech_par_id  
32675                  True      16       1907_16  
                                                  text  pred_class  \
388  For 175 years, through more than forty nationa..

...

       Unnamed: 0 Speech_id  \
15547       15548      2179   

                                                    text party    term  comp  \
15547  We live in a condition, as it once was describ...   rep  1968.0  True   

      populist_old_keywords  par_id speech_par_id  
15547                 False      29       2179_29  
                                                   text  pred_class  \
1265  We live in a condition, as it once was describ...           0   

     speech_par_id               speech_par_id_old  
1265       2179_29  2004-10-14-people-las-vegas_13  
       Unnamed: 0 Speech_id  \
19035       19036      2195   

                                                    text party    term   comp  \
19035  And now I am going to ask all the people in th...   rep  1968.0  False   

      populist_old_keywords  par_id speech_par_id  
19035                 False      15       2195_15  
                                                   text  pred_class  \
1283  And now I am goi

       Unnamed: 0 Speech_id  \
28296       28297      2602   

                                                    text party    term   comp  \
28296  In 1948, nobody thought Truman would win. But ...   dem  1980.0  False   

      populist_old_keywords  par_id speech_par_id  
28296                 False       3        2602_3  
                                                   text  pred_class  \
1502  In 1948, nobody thought Truman would win. But ...           0   

     speech_par_id speech_par_id_old  
1502        2602_3            2602_3  
      Unnamed: 0 Speech_id                                               text  \
2711        2712      2640   With your help, with your vote--and I need an...   

     party    term  comp populist_old_keywords  par_id speech_par_id  
2711   rep  1980.0  True                 False      16       2640_16  
                                                   text  pred_class  \
1527  With your help, with your vote--and I need and...           0   

 

      Unnamed: 0 Speech_id                                               text  \
6527        6528      2848  My opponent supported the grain embargo and sp...   

     party    term   comp populist_old_keywords  par_id speech_par_id  
6527   rep  1984.0  False                 False      21       2848_21  
                                                   text  pred_class  \
1677  My opponent supported the grain embargo and sp...           0   

     speech_par_id speech_par_id_old  
1677       2848_21           2848_21  
       Unnamed: 0 Speech_id  \
44928       44929      2855   

                                                    text party    term   comp  \
44928  Now, he says that he cares about the middle cl...   rep  1984.0  False   

      populist_old_keywords  par_id speech_par_id  
44928                 False      12       2855_12  
                                                   text  pred_class  \
1683  Now, he says that he cares about the middle cl...           0   


       Unnamed: 0 Speech_id  \
35812       35813      3040   

                                                    text party    term   comp  \
35812  My fellow Americans, I do not seek a victory o...   dem  1992.0  False   

      populist_old_keywords  par_id speech_par_id  
35812                 False      14       3040_14  
                                                   text  pred_class  \
1839  My fellow Americans, I do not seek a victory o...           0   

     speech_par_id speech_par_id_old  
1839       3040_14           3040_14  
       Unnamed: 0 Speech_id  \
32455       32456      3043   

                                                    text party    term  comp  \
32455  SENATOR HARKIN. Not then. GOVERNOR CLINTON. We...   dem  1992.0  True   

      populist_old_keywords  par_id speech_par_id  
32455                 False      19       3043_19  
                                                   text  pred_class  \
1843  SENATOR HARKIN. Not then. GOVERNOR CLINTON. 

       Unnamed: 0 Speech_id  \
13113       13114      3272   

                                                    text party    term  comp  \
13113  When we were fighting for the battle over affi...   dem  1996.0  True   

      populist_old_keywords  par_id speech_par_id  
13113                 False      10       3272_10  
                                                   text  pred_class  \
2041  When we were fighting for the battle over affi...           0   

     speech_par_id speech_par_id_old  
2041       3272_10           3272_10  
       Unnamed: 0 Speech_id  \
29732       29733      3318   

                                                    text party    term  comp  \
29732  It would have created fifty brand-new federal ...   rep  1996.0  True   

      populist_old_keywords  par_id speech_par_id  
29732                 False      16       3318_16  
                                                   text  pred_class  \
2096  It would have created fifty brand-new federal 